<a href="https://colab.research.google.com/github/selenamenezes/Farmacia/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import sqlite3
from datetime import datetime
conexao = sqlite3.connect('farmacia.db')
cursor = conexao.cursor()

cursor.execute("""
create table produtos(
  cod_barras integer primary key not null,
  nome text not null,
  data text not null,
  quantidade integer not null,
  preco real not null
)
""")

conexao.commit()
conexao.close()

In [11]:
def formatar_cod(cod_barras):
  if '-' in cod_barras or ' ' in cod_barras:
    cod_barras = cod_barras.replace(' ', '').replace('-', '')
  return cod_barras

def adicionar_produto(conexao, cod_barras, nome, data, quantidade, preco):
  cursor = conexao.cursor()
  cod = formatar_cod(cod_barras)
  quantidade = int(quantidade)
  preco = float(preco)

  cursor.execute("select * from produtos where cod_barras = ?", (cod, ))
  registro = cursor.fetchone()

  if len(cod) != 13:
    print("Código de barras inválido.")
    return

  if (quantidade < 0) or (preco < 0):
    print("Valores inválidos.")
    return

  if not registro:
    cursor.execute('''insert into produtos(cod_barras, nome, data, quantidade, preco)
                    values(?, ?, ?, ?, ?)''', (cod, nome, data, quantidade, preco))
    conexao.commit()

    if quantidade < 10:
      print("Baixo estoque!")
    print("Produto adicionado.")

  else:
    print("Produto já adicionado.")
    return

def listar_produtos(conexao):
  cursor = conexao.cursor()
  cursor.execute("select * from produtos")
  registro = cursor.fetchall()
  if registro:
    for col in registro:
      print("-= MEDICAMENTOS =-")
      print(f"{col[1]:<3} | cod. {col[0]} - Val: {col[2]} - {col[3]}un. - R$: {col[4]:.2f}")
      print("-" * 30)
  else:
    print("Não há produtos cadastrados.")

def atualizar_produto(conexao, cod_barras, nome, data, quantidade, preco):
  cursor = conexao.cursor()
  cod = formatar_cod(cod_barras)
  quantidade = int(quantidade)
  preco = float(preco)

  if (quantidade <= 0) or (preco <= 0):
    print("Valores inválidos.")
    return

  if len(cod) != 13:
    print("Código de barras inválido.")
    return

  cursor.execute("select * from produtos where cod_barras = ?", (cod, ))
  produto = cursor.fetchone()

  if produto:
    cursor.execute('''update produtos set nome = ?, data = ?, quantidade = ?, preco = ?
                        where cod_barras = ?''', (nome, data, quantidade, preco, cod))
    conexao.commit()

    if quantidade < 10:
      print("Baixo estoque!")
    print("Produto alterado!")

  else:
      print("Produto não encontrado.")

def remover_produto(conexao, cod_barras):
  cursor = conexao.cursor()
  cod = formatar_cod(cod_barras)
  cursor.execute("select * from produtos where cod_barras = ?", (cod, ))
  registro = cursor.fetchone()

  if len(cod) != 13:
    print("Código de barras inválido.")
    return

  if not registro:
    print("Não há produtos cadastrados.")
    return

  cursor.execute(''' delete from produtos where cod_barras = ?''', (cod, ))
  conexao.commit()
  print("Produto removido!")

def main():
  conexao = sqlite3.connect('farmacia.db')

  while True:
    menu = '''
    -= FARMÁCIA =-
    [1] Adicionar produto
    [2] Atualizar produto
    [3] Remover produto
    [4] Listar produtos
    [5] Sair
    >>> '''

    op = input(menu)

    if op == '1':
      print("-= Cadastro de Produto =-")
      nome = input("Nome: ")
      data = input("Data de Validade: ")
      preco = input("Preço: ")
      quantidade = input("Quantidade: ")
      cod_barras = input("Cod. Barras: ")
      adicionar_produto(conexao, cod_barras, nome, data, quantidade, preco)

    elif op == '2':
      print("-= Atualizar Produto =-")
      cod = input("Cod. Barras: ")
      nome = input("Nome: ")
      data = input("Data de Validade: ")
      preco = input("Preço: ")
      quantidade = input("Quantidade: ")
      atualizar_produto(conexao, cod, nome, data, quantidade, preco)

    elif op == '3':
      print("-= Remover Produto =-")
      cod = input("Cod. Barras: ")
      remover_produto(conexao, cod)

    elif op == '4':
      listar_produtos(conexao)

    elif op == '5':
      print("Saindo..")
      return

    else:
      print("Opção inválida.")

  conexao.close()

if __name__ == '__main__':
  main()



    -= FARMÁCIA =-
    [1] Adicionar produto
    [2] Atualizar produto
    [3] Remover produto
    [4] Listar produtos
    [5] Sair
    >>> 2
-= Atualizar Produto =-
Cod. Barras: 1234567890123
Nome: Simeticona
Data de Validade: 12/09/2030
Preço: 18.0
Quantidade: 9
Baixo estoque!
Produto alterado!

    -= FARMÁCIA =-
    [1] Adicionar produto
    [2] Atualizar produto
    [3] Remover produto
    [4] Listar produtos
    [5] Sair
    >>> 4
-= MEDICAMENTOS =-
Simeticona | cod. 1234567890123 - Data de Validade: 12/09/2030 - 9un. - R$: 18.00
------------------------------

    -= FARMÁCIA =-
    [1] Adicionar produto
    [2] Atualizar produto
    [3] Remover produto
    [4] Listar produtos
    [5] Sair
    >>> 5
Saindo..
